### Token coverage of data by trimmed models

In [ ]:
from machine.corpora import FileParatextProjectTextUpdater, FileParatextProjectSettingsParser, UsfmFileText, ScriptureRef, UsfmTextType
from pathlib import Path
from typing import List

In [ ]:
project_dir = Path("test_S/Paratext/projects/RVR1960")
book = "1JN"

src_settings = FileParatextProjectSettingsParser(project_dir).parse()
src_file_text = UsfmFileText(
    src_settings.stylesheet,
    src_settings.encoding,
    book,
    project_dir / src_settings.get_book_file_name(book),
    src_settings.versification,
    include_all_text=True,
    project=src_settings.name,
)

for s in src_file_text:
    print(type(s))
    break

# sentences = [s.text.strip() for s in src_file_text]
# vrefs: List[ScriptureRef] = [s.ref for s in src_file_text]

# for i in range(len(sentences)):
#     if vrefs[i].chapter_num == 3:
#         sentences[i] = ""

# trg_dir = Path("test_S/Paratext/projects/NIV11")
# dest_updater = FileParatextProjectTextUpdater(trg_dir)
# rows = [([ref], translation) for ref, translation in zip(vrefs, sentences)]
# usfm_out = dest_updater.update_usfm(
#     src_file_text.id, rows, strip_all_text=False, prefer_existing_text=False
# )
# with open("out.SFM", "w", encoding=src_settings.encoding) as f:
#     f.write(usfm_out)

In [ ]:
from transformers import AutoTokenizer

lang = "Rajbanshi"
data_fnames = ["align.src.detok.txt", "align.trg.detok.txt", "val.src.detok.txt", "val.trg.detok.txt", "test.src.detok.txt"]
data = []
for fname in data_fnames:
    # if "src" in fname:
    #     continue
    with open(f"zzz_data/{lang}/{fname}", "r", encoding="utf-8-sig") as f:
        data += [line.strip() for line in f.readlines()]
data_trg = []
with open(f"zzz_data/{lang}/test.trg.detok.txt", "r", encoding="utf-8-sig") as f:
    data_trg += [line.strip() for line in f.readlines()]

tokenizer = AutoTokenizer.from_prealigned("facebook/nllb-200-distilled-1.3B", use_fast=False)

data_ids = tokenizer(data, add_special_tokens=False)["input_ids"]
data_ids_trg = tokenizer(data_trg, add_special_tokens=False)["input_ids"]

In [ ]:
total_ids = 0
vocab = set()
for ids in data_ids:
    total_ids += len(ids)
    vocab.update(ids)

total_trg_ids = 0
vocab_trg = set()
for ids in data_ids_trg:
    total_trg_ids += len(ids)
    vocab_trg.update(ids)

raw_coverage = len(vocab_trg.intersection(vocab))
tokens_covered = 0
for ids in data_ids_trg:
    for id in ids:
        tokens_covered += id in vocab

print(total_ids, len(vocab))
print(total_trg_ids, len(vocab_trg))
print(raw_coverage, raw_coverage / len(vocab_trg))
print(tokens_covered, tokens_covered / total_trg_ids)


In [ ]:
import pandas as pd

verse_counts_df = pd.read_csv("verse_counts_orig.csv", index_col="file")
print(verse_counts_df)

# .sort_index().astype(int)
verse_counts_df.to_csv("verse_counts_mod.csv")